# VIME

In [ ]:
pip install pandas scikit-learn openml torch numpy

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.0/158.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 31.5 MB/s eta 0:00:00
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11716 sha256=be502ff6f4d15aa6a2052f803b67b545b78f433011c05edaf1565cffe665bb6c
  Stored in directory: /root/.cache/pip/wheels/5d/2a/9c/3895d9617f8f49a0883ba686326d598e78a1c2f54fe3cae86d
Successfully built liac-arff


In [ ]:
pip install torch torchvision torchaudio


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import openml

class VIMESelf(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, input_dim)
        self.mask_layer = nn.Linear(input_dim, input_dim)
        self.feat_layer = nn.Linear(input_dim, input_dim)

    def forward(self, x):
        x = F.relu(self.input_layer(x))
        out_mask = torch.sigmoid(self.mask_layer(x))
        out_feat = torch.sigmoid(self.feat_layer(x))
        return out_mask, out_feat


class VIMESemi(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=100, n_layers=5):
        super().__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.layers = nn.ModuleList(
            [nn.Linear(hidden_dim, hidden_dim) for _ in range(n_layers - 1)]
        )
        self.output_layer = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.input_layer(x))
        for layer in self.layers:
            x = F.relu(layer(x))
        out = self.output_layer(x)
        return out


class VIME:
    def __init__(self, params, args):
        self.params = params
        self.args = args
        self.model_self = VIMESelf(args['num_features']).to(args['device'])
        self.model_semi = VIMESemi(args['num_features'], args['num_classes']).to(args['device'])

        if args['data_parallel']:
            self.model_self = nn.DataParallel(self.model_self)
            self.model_semi = nn.DataParallel(self.model_semi)

    def fit(self, X, y, X_val=None, y_val=None):
        self.fit_self(X)
        return self.fit_semi(X, y, X_val, y_val)

    def fit_self(self, X):
        optimizer = optim.RMSprop(self.model_self.parameters(), lr=0.001)
        loss_func_mask = nn.BCELoss()
        loss_func_feat = nn.MSELoss()

        m_unlab = self.mask_generator(0.3, X)
        m_label, x_tilde = self.pretext_generator(m_unlab, X)

        x_tilde = torch.tensor(x_tilde).float().to(self.args['device'])
        m_label = torch.tensor(m_label).float().to(self.args['device'])
        X = torch.tensor(X).float().to(self.args['device'])
        train_dataset = TensorDataset(x_tilde, m_label, X)
        train_loader = DataLoader(train_dataset, batch_size=self.args['batch_size'], shuffle=True)

        for epoch in range(10):
            for batch_X, batch_mask, batch_feat in train_loader:
                out_mask, out_feat = self.model_self(batch_X)
                loss_mask = loss_func_mask(out_mask, batch_mask)
                loss_feat = loss_func_feat(out_feat, batch_feat)
                loss = loss_mask + loss_feat * self.params['alpha']

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

    def fit_semi(self, X, y, X_val=None, y_val=None):
        optimizer = optim.AdamW(self.model_semi.parameters())
        X = torch.tensor(X).float().to(self.args['device'])
        y = torch.tensor(y).long().to(self.args['device'])
        X_val = torch.tensor(X_val).float().to(self.args['device'])
        y_val = torch.tensor(y_val).long().to(self.args['device'])

        loss_func_supervised = nn.CrossEntropyLoss()

        train_dataset = TensorDataset(X, y)
        train_loader = DataLoader(train_dataset, batch_size=self.args['batch_size'], shuffle=True)

        for epoch in range(self.args['epochs']):
            for batch_X, batch_y in train_loader:
                y_hat = self.model_semi(batch_X)

                loss = loss_func_supervised(y_hat, batch_y)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        # Evaluation on validation set
        with torch.no_grad():
            val_preds = self.model_semi(X_val)
            val_preds = torch.argmax(val_preds, dim=1)
            accuracy = (val_preds == y_val).float().mean().item()
            return accuracy

    @staticmethod
    def mask_generator(p_m, x):
        mask = np.random.binomial(1, p_m, x.shape)
        return mask

    @staticmethod
    def pretext_generator(m, x):
        no, dim = x.shape
        x_bar = np.zeros([no, dim])
        for i in range(dim):
            idx = np.random.permutation(no)
            x_bar[:, i] = x[idx, i]

        x_tilde = x * (1 - m) + x_bar * m
        m_new = 1 * (x != x_tilde)
        return m_new, x_tilde

def load_openml_data(task_id):
    task = openml.tasks.get_task(task_id)
    dataset = task.get_dataset()
    X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute)

    # Identify categorical columns (exclude numeric columns)
    categorical_columns = X.select_dtypes(include=['object', 'category']).columns

    # Apply One-Hot Encoding to categorical columns
    X = pd.get_dummies(X, columns=categorical_columns, drop_first=True)

    # Handle non-finite values by filling NaNs with zeros (or use an appropriate strategy for your case)
    X.replace([np.inf, -np.inf], np.nan, inplace=True)
    X.fillna(0, inplace=True)

    # Ensure all columns in X are numeric
    X = X.apply(pd.to_numeric, errors='coerce')
    X.fillna(0, inplace=True)

    # Check for non-numeric columns
    non_numeric_columns = X.select_dtypes(exclude=['number']).columns
    if len(non_numeric_columns) > 0:
        # Drop non-numeric columns
        X = X.drop(columns=non_numeric_columns)

    # Convert y to numeric if it's categorical
    if y.dtype == 'object' or isinstance(y.dtype, pd.CategoricalDtype):
        y = pd.factorize(y)[0]

    # Ensure y is a numpy array and has the correct type
    y = np.asarray(y).astype(np.int64)

    # Final check for non-numeric types in X
    if not np.issubdtype(X.values.dtype, np.number):
        raise ValueError("There are still non-numeric columns in the dataset after conversion.")

    return X.to_numpy(), y


# Task ID1: 14965

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 14965  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.8857


# Task ID2: 9977

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 9977  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9479


# Task ID3: 34539

In [ ]:
# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 34539  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Validation Accuracy: 0.9437


# Task ID4: 146606

In [ ]:
# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 146606  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.6869


# Task ID5: 7592



In [ ]:
# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 7592  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.7945


# Task ID6: 146195



In [ ]:
# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 146195  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Validation Accuracy: 0.6569


# Task ID7: 167119



In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 167119 # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.8648


# Task ID8: 167120



In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 167120  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.5195


# Task ID11: 	168331



In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 168331  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.6197


# Task ID12: 	168330



In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 168330  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.6627


# Task ID13: 	168335



In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 168335  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9218



# Task ID16: 	146212



In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 146212  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9977


# Task ID19: 	168868



In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 168868  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9803


# Task ID20: 31

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 31  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.6800


# Task ID21: 10101

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 10101  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.7533


# Task ID22: 	3913

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 3913  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.7619


# Task ID23: 3

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 3  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Validation Accuracy: 0.5312


# Task ID24: 3917

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 3917  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.8436


# Task ID25: 9957

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 9957  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.8673


# Task ID26: 9946

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 9946  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9649


# Task ID27: 3918

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 3918  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9144


# Task ID28: 3903

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 3903  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.8978


# Task ID29: 37

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 37  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.7403


# Task ID30: 9971

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 9971  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.7436


# Task ID31: 9952

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 9952  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.8511


# Task ID32: 3902

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 3902  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.7842


# Task ID33: 49

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 49  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Validation Accuracy: 0.6510


# Task ID34: 43

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 43  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9338


# Task ID35: 9978

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 9978  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9349


# Task ID36: 10093

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 10093  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 1.0000


# Task ID37: 219

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 219  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.7723


# Task ID38: 9976

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 9976  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.4942


# Task ID39: 6

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 6  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9143


# Task ID40: 53

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 53  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.5176


# Task ID41: 11

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 11  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9840


# Task ID42: 15

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 15  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9857


# Task ID43: 16

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 16  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9725


# Task ID44: 14

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 14  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.6975


# Task ID45: 32

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 32  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9914


# Task ID46: 3549

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id =  3549  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9941


# Task ID47: 12

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 12  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.8675


# Task ID48: 9981

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 9981  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9398


# Task ID49: 18

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 18  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.3050


# Task ID50: 28

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 28  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9564


# Task ID51: 2074

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 2074  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9020


# Task ID52: 29

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 29  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.6304


# Task ID53: 45

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 45  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Validation Accuracy: 0.5000


# Task ID54: 125922

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 125922  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9164


# Task ID55: 9960

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 9960  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.8718


# Task ID56: 9964

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 9964  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.8934


# Task ID57: 22

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 22  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.8325


# Task ID58: 2079

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 2079  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.4730


# Task ID59: 14969

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 14969  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.4678


# Task ID60: 3560

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 3560  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Validation Accuracy: 0.2438


# Task ID61: 14952

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 14952  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Validation Accuracy: 0.5676


# Task ID62: 125920

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 125920  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.5800


# Task ID63: 23

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 23  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.4949


# Task ID64: 3904

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 3904  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.8066


# Task ID65: 3022

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 3022  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.6970


# Task ID66: 9985

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 9985  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.5245


# Task ID67: 9910

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 9910  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.7816


# Task ID68: 14970

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 14970  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9107


# Task ID69: 3021

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 3021  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9497


# Task ID70: 3481

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 3481  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9064


# Task ID71: 3573

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 3573  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


# Task ID72: 146824

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 146824  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9675


# Task ID73: 146820

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 146820  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9669


# Task ID74: 146822

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 146822  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.8745


# Task ID75: 146195

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 146195  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Validation Accuracy: 0.6569


# Task ID76: 146800

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 146800  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.5093


# Task ID77: 146817

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 146817  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.3753


# Task ID78: 146819

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 146819  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.9074


# Task ID79: 146821

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 146821 # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Validation Accuracy: 0.6792


# Task ID80: 14954

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 14954  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.4444


# Task ID81: 167141

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 167141 # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.8650


# Task ID82: 167140

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 167140  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Validation Accuracy: 0.4969


# Task ID83: 167125

In [ ]:


# Main execution
if __name__ == "__main__":
    # Specify OpenML task ID
    task_id = 167125  # Replace with your desired task ID
    X, y = load_openml_data(task_id)

    # Determine number of features and classes from the dataset
    num_features = X.shape[1]
    num_classes = len(np.unique(y))

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define parameters and arguments
    params = {
        "p_m": 0.5,
        "alpha": 3.0,
    }
    args = {
        "num_features": num_features,
        "num_classes": num_classes,
        "batch_size": 32,
        "epochs": 10,
        "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        "data_parallel": False,
    }

    # Initialize and train VIME model
    model = VIME(params, args)
    accuracy = model.fit(X_train, y_train, X_val, y_val)

    # Print only validation accuracy
    print(f"Validation Accuracy: {accuracy:.4f}")


Validation Accuracy: 0.8933


# Task ID84: 167124

# Task ID85: 167121

In [ ]:
pip freeze > requirements.txt